In [23]:
include("compute_ph2.jl")
using LIBSVM
using RDatasets
using Printf
using Statistics
using Random
using .ext
using Plots
using GLM
using StatsBase
using ScikitLearn
@sk_import model_selection: train_test_split

┌ Warning: Module model_selection has been ported to Julia - try `import ScikitLearn: CrossValidation` instead
└ @ ScikitLearn.Skcore C:\Users\jaydh\.julia\packages\ScikitLearn\ssekP\src\Skcore.jl:179


PyObject <function train_test_split at 0x0000000097C318B0>

In [24]:
x = [i for i = 1:10:1620]
y = [j for j = 6000:10:7100]
ids = vcat(x,y)
times = [250,300,350,400,450,500]
sims = [[id,time] for id in ids for time in times];
celltype1 = "Vessel";
celltype2 = "Macrophage";
sims = cleanSimsDowker(sims,celltype1,celltype2);
SIZE = 15;
VAR = 1;

In [42]:
sims

1188-element Vector{Vector{Int64}}:
 [1, 250]
 [1, 300]
 [1, 350]
 [1, 400]
 [1, 450]
 [1, 500]
 [11, 250]
 [11, 300]
 [11, 350]
 [11, 400]
 [11, 450]
 [11, 500]
 [21, 250]
 ⋮
 [7080, 250]
 [7080, 300]
 [7080, 350]
 [7080, 400]
 [7080, 450]
 [7080, 500]
 [7090, 250]
 [7090, 300]
 [7090, 350]
 [7090, 400]
 [7090, 450]
 [7090, 500]

In [25]:
labels = getBinaryLabels(sims);
 # gets labels for each sim. 
 # 1 if M2/(M1+M2) > 0.5. 
 # 0 Otherwise. 
 # Thresholding phenotypes < 0.5 anti-tumour, >= 0.5 pro-tumour
features = getDowkerfeatures(0, sims, celltype1, celltype2, SIZE, VAR);
 # gets feature vectors for chosen method. Outputs matrix of size N x M 
 # N = number of sims
 # M = size of feature vector, i.e. M = SIZE*SIZE where SIZE = size of persistence image
 # e.g. 1188 sims considered, calculate 20x20 persistence images for each --> 1188x400 matrix

In [26]:
df = DataFrame(sims = sims, labels = labels)
for i = 1:(SIZE*SIZE)
    colname = string("feature_",i)
    df[!,colname] = features[:,i]
end
# create dataframe with simulations, labels, and all features

In [27]:
df

,sims,labels,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6
,Array…,Any,Float64,Float64,Float64,Float64,Float64,Float64
1,"[1, 250]",0,0.00740886,0.020425,0.040385,0.0859821,0.101502,0.0544513
2,"[1, 300]",0,0.0174308,0.0376076,0.0687476,0.0985676,0.0817153,0.0403405
3,"[1, 350]",0,0.0157873,0.0467754,0.0830455,0.0968372,0.0567234,0.0225275
4,"[1, 400]",0,0.0226509,0.0499479,0.0740949,0.0785724,0.0438015,0.0223333
5,"[1, 450]",0,0.0301554,0.0786518,0.109992,0.0793434,0.0365893,0.0148492
6,"[1, 500]",0,0.035206,0.0791842,0.100837,0.0794835,0.0382706,0.00944709
7,"[11, 250]",0,0.013138,0.0357185,0.0585445,0.0917087,0.096745,0.0623645
8,"[11, 300]",0,0.0190491,0.0386717,0.060178,0.0828999,0.0776181,0.0457038
9,"[11, 350]",0,0.0236567,0.0532344,0.0775518,0.0899351,0.0629278,0.0188073


In [28]:
# random undersampling 

class_0 = countmap(df.labels)[0]
class_1 = countmap(df.labels)[1]

Random.seed!(1);
df=df[shuffle(1:nrow(df)),:];   

if class_0 > class_1 
    # randomly undersample class 0

    prob_to_delete = (class_0-class_1)/(class_0)
    rows_to_delete = []
    for row in 1:length(df.labels)
        if df[row,:labels] == 0
            if rand() < prob_to_delete
                append!(rows_to_delete,row)
            end
        end
    end
    delete!(df,rows_to_delete)
elseif class_1 > class_0 
    # randomly undersample class 1

    prob_to_delete = (class_1-class_0)/(class_1)
    rows_to_delete = []
    for row in 1:length(df.labels)
        if df[row,:labels] == 1
            if rand() < prob_to_delete
                append!(rows_to_delete,row)
            end
        end
    end
    delete!(df,rows_to_delete)
end

,sims,labels,feature_1,feature_2,feature_3,feature_4,feature_5
,Array…,Any,Float64,Float64,Float64,Float64,Float64
1,"[951, 500]",0,0.011055,0.0230288,0.0252146,0.0153139,0.0127972
2,"[651, 450]",1,0.0145121,0.0130357,0.0150536,0.0291809,0.0265747
3,"[901, 250]",0,1.98983e-9,7.00509e-7,0.00024704,0.00665223,0.0187105
4,"[1371, 250]",1,5.55306e-24,2.45581e-23,2.4084e-23,5.05982e-24,9.5356e-26
5,"[1151, 350]",1,0.00296121,0.00619825,0.0107698,0.00703784,0.000701231
6,"[821, 400]",1,0.00421397,0.0187837,0.0296966,0.0199009,0.00742934
7,"[6410, 450]",1,0.00125018,0.00613746,0.0113962,0.0179503,0.0253942
8,"[1151, 450]",1,0.00616752,0.0121388,0.016132,0.0236991,0.0178399
9,"[771, 450]",0,0.00141156,0.00124019,0.00550951,0.013438,0.0220592


In [29]:
countmap(df.labels)

Dict{Any, Int64} with 2 entries:
  0 => 272
  1 => 269

In [38]:
# randomly sample training, test and validation sets

Random.seed!(1);
df=df[shuffle(1:nrow(df)),:];   

X = Matrix(df[:,3:end]);    
y = df.labels;  

train_ratio = 0.6
validation_ratio = 0.2
test_ratio = 0.2
Xtrain, Xtest, ytrain, ytest = train_test_split(X,y,test_size=1-train_ratio); # train-test split
Xval, Xtest, yval, ytest = train_test_split(Xtest,ytest,test_size= (test_ratio)/(test_ratio+validation_ratio)); # test-validation split

In [39]:
# parameter optimisation on training and validation set 
# could add search over more parameters too
gammas = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
costs = gammas
Cs = [] 
Gs = []
max = 0
for i=1:1:7
    G = gammas[i]
    for j=1:1:7
        C = costs[j]
        model = svmtrain(Xtrain', ytrain ; gamma=G , cost=C)
        ypredict, decision_values = svmpredict(model, Xval');
        acc = mean(ypredict .== yval) * 100;
        if acc > max 
            #println("current max: ",acc )
            max = acc
            append!(Cs,C)
            append!(Gs,G)
        end
    end
end

In [40]:
println("Cost: ", Cs[end])
println("Gamma: ", Gs[end])

Cost: 1.0
Gamma: 100.0


In [41]:
# report final accuracy on test set

model = svmtrain(Xtrain', ytrain ; gamma=Gs[end] , cost=Cs[end])

ypredict, decision_values = svmpredict(model, Xtest');
acc = mean(ypredict .== ytest) * 100;
println("accuracy: ", acc)

accuracy: 78.89908256880734
